Importamos las librerías (obviamente voy a ir añadiendo librerías en base vaya creciendo el código)

In [1]:
import os
import torch
import shutil
from torch import nn
from torch.nn import functional as f
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image, UnidentifiedImageError
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv(r'..\..\labels\labels.csv')
df

,filename,label
0,0.jpg,Cat
1,1.jpg,Cat
2,10.jpg,Cat
3,1000.jpg,Cat
4,10000.jpg,Cat
...,...,...
18321,7995.jpg,Dog
18322,7996.jpg,Dog
18323,3737.jpg,Dog
18324,8588.jpg,Dog


El dataset en sí mismo tiene "Filename collision" gracias a explorar las carpetas de imágenes, pero bueno ¿Qué quiere decir este término? Bueno, el término **Filename collision** refiere a que distintas imágenes tiene el mismo nombre, en este caso accederemos a la imagen "0.jpg", lo normal sería que solo diera un resultado, pero como podremos ver, no es el caso.

In [3]:
df[df['filename'] == '0.jpg']

,filename,label
0,0.jpg,Cat
8270,0.jpg,Dog


Como podemos ver, distintas imágenes tienen el mismo nombre '0.jpg', lo que puede causar problemas a la hora de entrenar el modelo, por ejemplo, si en el dataframe carga el archivo 0.jpg, Cat, pero el modelo (el DataLoader) carga 0.jpg, Dog, puede haber mala generalización, confusiones del modelo y resultados aleatorios.

Para resolver esto, pensé en renombrar los archivos en una misma carpeta, pero no perdería los datos, porque me basaré en sus índices para crear sus nuevos nombres, sin perder la etiqueta que son.

Primero realizamos el reseteo de índices del DataFrame con el parámetro drop en True para evitar que el índice que tiene, se convierta en una columna

In [4]:
df = df.reset_index(drop=True)

"""Una vez tenemos los índices, podemos crear los nombres, para esto creamos una columna/campo llamado "filename_nuevo" que su valor es el método index 
del DataFrame para obtener su índice y usamos .astype('str') para obtenerlo como string y a eso le añadimos el tipo de archivo, en este caso JPG"""

df['filename_nuevo'] = df.index.astype(str) + '.jpg'
df

,filename,label,filename_nuevo
0,0.jpg,Cat,0.jpg
1,1.jpg,Cat,1.jpg
2,10.jpg,Cat,2.jpg
3,1000.jpg,Cat,3.jpg
4,10000.jpg,Cat,4.jpg
...,...,...,...
18321,7995.jpg,Dog,18321.jpg
18322,7996.jpg,Dog,18322.jpg
18323,3737.jpg,Dog,18323.jpg
18324,8588.jpg,Dog,18324.jpg


Como podemos observar, ya tenemos los datos con un solo nombre, evitando esto el problema "Filename collision", ahora debemos copiarlo a la carpeta donde estarán todos los archivos, esto se hace con la librería os para los paths y shutil para mover los archivos.

Primero creamos la ruta base donde se encuentran todas las carpetas y sus imágenes: ..\ ..\PetImages

Después debemos iterar sobre las filas del DataFrame, lo cual podemos realizarlo con el método iterrows() del DataFrame y un bucle for con tuplas (idx, fila)

In [5]:
ruta_base = r'..\..\PetImages'

for idx, fila in df.iterrows():
    #Ahora debemos obtener el label, el anterior nombre y el nuevo nombre que cree con sus índices
    label = fila['label']
    old_name = fila['filename']
    new_name = fila['filename_nuevo']

    #Ahora con la anterior información podemos crear los paths, donde imaginemos que label = Dog, old_name='0.jpg' y el nuevo nombre es 10.jpg
    old_path = os.path.join(ruta_base, label, old_name) # Resultado: C:\Users\PC\Desktop\Road_to_Hackathon\PetImages\Dog\0.jpg
    dir_path = os.path.join(ruta_base, 'renamed_ds')  # Resultado: C:\Users\PC\Desktop\Road_to_Hackathon\PetImages\renamed_ds (Esta carpeta la llame así porque me quedé sin ideas jaja)
    new_path = os.path.join(ruta_base, 'renamed_ds', new_name) # Resultado: C:\Users\PC\Desktop\Road_to_Hackathon\PetImages\renamed_ds\10.jpg
    
    #Ahora necesitamos hacer una condición para saber si es que existe o no el directorio renamed_ds (esto solo pa' la primera vez que ejecuto el código jajajsfaj)
    if not os.path.isdir(dir_path): # 1era ejecución: False -1: True lmao xd
        #Ahora creamos el directorio
        os.mkdir(dir_path)
    
    #Ahora copiamos el archivo
    shutil.copy(old_path, new_path)

print("Los archivos han sido copiados")


Los archivos han sido copiados


![Imagen de la carpeta renamed_ds](../../Images/Random/folder_renamedDS.png)

Ala, aparcao' ya que tenemos todos los archivos movidos, solo queda limpiar el dataframe para pasarlo a el slice de dataset para entrenamiento y validación, solo para corroborar que no haya Filename collision ejecutamos el siguiente bloque de código:

In [6]:
df[df['filename_nuevo'] == '0.jpg']

,filename,label,filename_nuevo
0,0.jpg,Cat,0.jpg


Como podemos observar todavía debemos borrar la columna que podría llegar a colisiones de nombre, pero tomando como argumento la columna del DataFrame "filename_nuevo"

In [7]:
#Ahora borramos la columna 'filename' que tiene el problema del Filename collision, con el parámetro: inplace con el valor de True, para que modifique el DataFrame y no cree una copia.
df.drop(columns=['filename'], inplace=True)

In [8]:
#Renombramos la columna filename_nuevo a filename
df.rename(columns={"filename_nuevo":'filename'}, inplace=True)

In [9]:
#Creamos una lista con los nombres de las columnas del DataFrame
columnas = df.columns.tolist()

#Quitamos el campo/columna filename
columnas.remove('filename')

In [10]:
#Nomás pa' ver las columnas en la lista XD
columnas

['label']

In [11]:
#Insertamos en la posición 0 el argumento 'filename'
columnas.insert(0, 'filename')

In [12]:
#Volvemos a mostrar, ala aparcao' de nuevo pa' poder ponerlas en el DataFrame
columnas

['filename', 'label']

In [13]:
#Renombramos las columnas del DataFrame y lo mostramos pa' corroborar
df = df[columnas]
df

,filename,label
0,0.jpg,Cat
1,1.jpg,Cat
2,2.jpg,Cat
3,3.jpg,Cat
4,4.jpg,Cat
...,...,...
18321,18321.jpg,Dog
18322,18322.jpg,Dog
18323,18323.jpg,Dog
18324,18324.jpg,Dog


In [14]:
#Ahora si ejecutamos el método que ya teníamos anteriormente, pero con el argumento filename porque ya renombramos el.
df[df['filename'] == '0.jpg']

,filename,label
0,0.jpg,Cat


Ahora que tenemos el DataFrame, podemos crear la clase para crear el Dataset customizado para el modelo que entrenaremos. 

El código es similar al que nos otorga la documentación de PyTorch, donde como instancias declaramos 4 la cual es el **DataFrame**, el **directorio de imágenes**, **transformador de imágenes** (normalizaciones, convertirlo a tensor de PyTorch, DataAugmentation, etc.) y **transformador de etiquetas** (convertir de one-hot a label encoding o de label encoding a logits [probabilidades])

In [15]:

class CustomDS(Dataset):
    def __init__(self, dataframe, images_dir, transform=None, target_transform=None): #Parámetros modificables
        self.labels = dataframe #Le pasamos el DataFrame que ya habíamos hecho sin el problema de Filename collision
        self.directory = images_dir #Ruta al directorio de imágenes
        self.transform_data = transform #Transformador de la imagen
        self.target_transform = target_transform #Transformador de etiquetas
        self.mapping = {"Cat": 0, "Dog": 1} #Mapeo de etiquetas

    def __len__(self):
        return len(self.labels) #Solamente obtiene la cantidad de elementos

    def __getitem__(self, idx):
        label_df = self.labels.iloc[idx, 1] #Obtiene el elemento de la fila (supongamos idx = 0) 0 y columna 1, lo que sería el elemento de la posición [0, 1] si vieramos al DataFrame como una matriz
        img_path = os.path.join(self.directory, self.labels.iloc[idx, 0]) #Generamos el path de donde se encuentran las imágenes (Notese que solo accedo al elemento, porque ya no hay distinción entre imágenes hablando de las carpetas)
        img = Image.open(img_path).convert('RGB') #Abrimos la imagen con PIL para saber si la imagen no está corrupta o dañada y si esto es cierto, la convertimos a RGB para tener 3 canales en el tensor
        img_label = self.mapping[label_df] #Mapeamos la etiqueta, es decir, si la etiqueta es Cat, la convierte en 0 o si es Dog la convierte en 1
        if self.transform_data: #Revisamos si se pasaron transformadores para los datos
            img = self.transform_data(img)
        if self.target_transform: #Lo mismo, pero para etiquetado
            img_label = self.target_transform(img_label)
        return img, img_label #Retornamos la imagen y su etiqueta

Solo quiero mencionar dos cosas que no mencioné en el código anterior, el código anterior en la sección de "transformador de etiquetas" me refiero al hecho que la etiqueta, puede permanecer como un entero o como un tensor donde ese tensor tiene el formato de **One-hot encoding** ¿A qué hace referencia este término? Es simplemente que si tenemos por ejemplo... la etiqueta 0, este número entero se convierte a un tensor donde se marca la posición 0 del tensor como un 1 y la demás 0 para determinar que es un 0, algo más gráfico sería:

0 -> torch.nn.functional.one_hot(torch.Tensor(label), num_classes=2) -> tensor([1, 0]) //La posición 0 del tensor marcado por el número 1, es el que determina a qué clase pertenece.

Además el otro punto que quería mencionar es que al momento en el que se cargan los archivos, es donde el problema de **Filename collision** es el que podría generar el problema de carga distinta imagen a la que el etiquetado sugiere. Para ser más claros pondré un ejemplo:

Imaginemos que el modelo carga la imagen 0.jpg de la etiqueta "Gato", pero el modelo carga la otra imagen de la etiqueta "Perro" lo que causa que el modelo aprenda al revés el generalizar las imágenes

In [16]:
#Ahora debemos dividir el DataFrame con train_test_split de Scikit-learn (Opté por usar dicho método de Scikit porque ya esta optimizado y además es más sencillo de usar [no debo reinventar la rueda])
#con 20% de los datos para el dataset de validación y 80% para entrenar el modelo, además dividiremos el dataset de validación, para obtener una métrica más confiable con imágenes que nunca ha visto

train_df, val_df = train_test_split(
    df,
    test_size = 0.2,
    stratify=df['label'],
    random_state=788
)

val_df, dev_df = train_test_split(
    val_df,
    test_size=0.1,
    stratify=val_df['label'],
    random_state=788
)

Una vez tenemos los DataFrames de entrenamiento, validación y prueba / dev (asi lo llaman coloquialmente xd), podemos ahora si crear una instancia de cada uno de los 3 datasets para el modelo, pero antes revisemos los tamaños de todos ellos.

In [17]:
general_path = r'C:\Users\PC\Desktop\Road_to_Hackathon\PetImages\renamed_ds'

print(f"Train size: {len(train_df)}, Test size: {len(val_df)}, Dev size: {len(dev_df)}")

transformador = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

#Generación de las instancias de las clases CustomDS
train_dataset = CustomDS(dataframe=train_df, images_dir=general_path, transform=transformador)
val_dataset = CustomDS(dataframe=val_df, images_dir=general_path, transform=transformador)
dev_dataset = CustomDS(dataframe=dev_df, images_dir=general_path, transform=transformador)

#Generción de los DataLoaders
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=64, shuffle=True)
dev_dataloader = DataLoader(dev_dataset, batch_size=64, shuffle=True)

Train size: 14660, Test size: 3299, Dev size: 367


Una vez tenemos los datos (aún falta limpieza, pero ahorita más adelante lo resolvemos), generamos la arquitectura del modelo, pensé en el concepto de ResNet de usar dos convoluciones antes de pasar por un Pooling, solo que aquí opte por cortalo mediante un stride, algo que en el modelo final pienso cambiarlo por un MaxPooling porque para este problema solo debemos saber qué es, no debemos saber donde está y qué es (en ese caso stride = 2 está mejor que MaxPooling) ya que MaxPooling hace que se enfoque más en el objeto que más activaciones tuvo que en el que menos tuvo, filtrando algo de ruido de la imagen

In [18]:
#Aquí determinamos el dispositivo donde se harán todos los cálculos, en mi caso es una RTX 5070 TI TUF GAMING con 16 GB de VRam, pero optimizada para entrenamiento de modelos de Deep Learning por parte de NVIDIA
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#Generamos la clase del modelo con la herencia de nn.Module
class Modelo(nn.Module):
    def __init__(self):
        super(Modelo, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=(3, 3), padding = 1) #Input = 128, 128, 3
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=(3, 3)) #63
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=(3, 3), padding = 1)
        self.conv4 = nn.Conv2d(in_channels= 32, out_channels=64, kernel_size=(3, 3)) #31
        self.conv5 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=(3, 3), padding = 1)
        self.conv6 = nn.Conv2d(in_channels=128, out_channels=128, kernel_size=(3, 3)) #15
        self.conv7 = nn.Conv2d(in_channels=128, out_channels=128, kernel_size=(3, 3)) #13
        self.conv8 = nn.Conv2d(in_channels=128, out_channels=128, kernel_size=(3, 3)) #6
        self.maxpool = nn.MaxPool2d((2, 2))

        self.avg = nn.AdaptiveAvgPool2d((1, 1)) # batch, 128, 1, 1

        self.dense1 = nn.Linear(in_features=128, out_features=64)  
        self.dense2 = nn.Linear(in_features=64, out_features=2)

    def forward(self, x):
        x = f.relu(self.conv1(x))
        x = f.leaky_relu(self.maxpool(self.conv2(x)))
        x = f.relu(self.conv3(x))
        x = f.leaky_relu(self.maxpool(self.conv4(x)))
        x = f.relu(self.conv5(x))
        x = f.leaky_relu(self.maxpool(self.conv6(x)))
        x = f.relu(self.conv7(x))
        x = f.leaky_relu(self.maxpool(self.conv8(x)))

        x = self.avg(x)
        
        x = torch.flatten(x, 1)
        x = f.relu(self.dense1(x))
        x = self.dense2(x)

        return x

model = Modelo().to(device)
model.parameters

<bound method Module.parameters of Modelo(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (conv5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv6): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1))
  (conv7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1))
  (conv8): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1))
  (maxpool): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (avg): AdaptiveAvgPool2d(output_size=(1, 1))
  (dense1): Linear(in_features=128, out_features=64, bias=True)
  (dense2): Linear(in_features=64, out_features=2, bias=True)
)>

In [19]:

function_loss = nn.CrossEntropyLoss()
optimizador = torch.optim.Adam(model.parameters(), lr=1e-4)

In [20]:
#Ahora crearemos una función para limpiar datos corruptos o dañados usando la librería PIL con la clase Image y métodos open, para quitar el warning "UserWarning: Truncated File Read warnings.warn(str(msg))"
def clean_data(DirPath, DataFrame=None):
    DatosInaccesibles = []
    for _, row in DataFrame.iterrows():
        path = os.path.join(DirPath, row['filename'])
        try:
            with Image.open(path) as img:
                img.verify()
            with Image.open(path) as img:
                img.load()
        except (UnidentifiedImageError, OSError, ValueError) as ex:
            DatosInaccesibles.append(path)        
    return DatosInaccesibles

dictInaccesibles={}

ds_dir = r'..\..\PetImages'
df_dog = pd.read_csv(r'..\..\Labels\labels_Dog.csv')
df_cat = pd.read_csv(r'..\..\Labels\labels_Cat.csv')

print(df_dog, df_cat)

data_to_clean = [(os.path.join(ds_dir, 'Dog'), df_dog), (os.path.join(ds_dir, 'Cat'), df_cat)]

for path, dataframe in data_to_clean:
    print(path)
    dir_name = os.path.basename(path)
    dictInaccesibles[dir_name] = clean_data(DirPath=path, DataFrame=dataframe)

print(f"Archivos inaccesibles o dañados encontrados: {dictInaccesibles}")

        filename  label
0          0.jpg    Dog
1          1.jpg    Dog
2         10.jpg    Dog
3      10000.jpg    Dog
4      10001.jpg    Dog
...          ...    ...
10051   9994.jpg    Dog
10052   9995.jpg    Dog
10053   9996.jpg    Dog
10054   9997.jpg    Dog
10055   9998.jpg    Dog

[10056 rows x 2 columns]        filename  label
0         0.jpg    Cat
1         1.jpg    Cat
2        10.jpg    Cat
3      1000.jpg    Cat
4     10000.jpg    Cat
...         ...    ...
8265   9993.jpg    Cat
8266   9994.jpg    Cat
8267   9996.jpg    Cat
8268   9998.jpg    Cat
8269   9999.jpg    Cat

[8270 rows x 2 columns]
..\..\PetImages\Dog
..\..\PetImages\Cat
Archivos inaccesibles o dañados encontrados: {'Dog': [], 'Cat': []}


In [21]:
#Este fragmento lo hice para saber que imagen daba un warning y podría estar causando problemas de loss inestable
df_dog[df_dog['filename'] == '9040.jpg'].index

Index([9193], dtype='int64')

In [22]:
class customDSAlone(Dataset):
    def __init__(self, dataframe, img_path, transform = None, target_transform = None):
        self.dataframe = dataframe
        self.img_dir = img_path
        self.transform = transform
        self.target_transform = target_transform
        self.mapping = {'Cat': 0, 'Dog':1}
    
    def __len__(self):
        return len(self.dataframe)
    
    def __getitem__(self, idx):
        label = self.dataframe.iloc[idx, 1]
        _filename = self.dataframe.iloc[idx, 0]
        path_img = os.path.join(self.img_dir, label, _filename)
        image = Image.open(path_img).convert('RGB')
        mapped_label = self.mapping[label]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, mapped_label, _filename

In [23]:
DogDF = pd.read_csv(r'..\..\Labels\labels_Dog.csv')
CatDF = pd.read_csv(r'..\..\Labels\labels_Cat.csv')
DogDS = customDSAlone(dataframe=DogDF, img_path=r'..\..\Petimages', transform=transformador)
CatDS = customDSAlone(dataframe=CatDF, img_path=r'..\..\Petimages', transform=transformador)
DogDL = DataLoader(dataset = DogDS, batch_size=64, shuffle=True)
CatDL = DataLoader(dataset = CatDS, batch_size=64, shuffle=True)

In [24]:

def MismatchLabeling(model, dataloader, device, FunctionLoss):
    val_loss = 0.0
    total_samples = len(dataloader.dataset)
    correct = 0
    model.eval()
    model = model.to(device)
    resultados = []
    with torch.no_grad():
        for images, labels, filename in dataloader:
            images, labels = images.to(device), labels.to(device)

            outputs = model(images) #Aquí devuelve logits, por ejemplo si es solo 2 características (sería...) [2.1, 3.7]
            loss = FunctionLoss(outputs, labels) * images.size(0) #Acá se usa images.size(0) porque si tenemos batch = 32, pero con un loss 0.2, eso es el promedio, si quieres el general del batch, sería a la inversa, multiplicar por el total del batch
            val_loss += loss.item()
            prob = torch.nn.functional.softmax(outputs, 1)
            confianza, predict = torch.max(prob, 1)
            for i in range(images.size(0)):
                resultados.append({
                    'filename': filename[i],
                    'true_label': labels[i].item(),
                    'predicted_label': predict[i].item(),
                    'max_prob': confianza[i].item()
                })
                if labels[i] == predict[i]:
                    correct += 1
    val_loss /= len(dataloader.dataset)
    precision = (correct / total_samples) * 100
    return (f'El valor de pérdida del mdoelo es: {val_loss:.4f}, con una precisión de: {precision:.4f}', resultados)

In [25]:
Asd = MismatchLabeling(model = model, dataloader = DogDL, device = device, FunctionLoss = function_loss)
text, results = Asd

In [26]:
print(text)
print(results)

El valor de pérdida del mdoelo es: 0.7714, con una precisión de: 0.0000
[{'filename': '1343.jpg', 'true_label': 1, 'predicted_label': 0, 'max_prob': 0.537641167640686}, {'filename': '5679.jpg', 'true_label': 1, 'predicted_label': 0, 'max_prob': 0.537649393081665}, {'filename': '5577.jpg', 'true_label': 1, 'predicted_label': 0, 'max_prob': 0.537658154964447}, {'filename': '5377.jpg', 'true_label': 1, 'predicted_label': 0, 'max_prob': 0.5376369953155518}, {'filename': '5974.jpg', 'true_label': 1, 'predicted_label': 0, 'max_prob': 0.5376250147819519}, {'filename': '4681.jpg', 'true_label': 1, 'predicted_label': 0, 'max_prob': 0.5376365184783936}, {'filename': '12083.jpg', 'true_label': 1, 'predicted_label': 0, 'max_prob': 0.5376410484313965}, {'filename': '7057.jpg', 'true_label': 1, 'predicted_label': 0, 'max_prob': 0.5376206636428833}, {'filename': '10334.jpg', 'true_label': 1, 'predicted_label': 0, 'max_prob': 0.5376660823822021}, {'filename': '1467.jpg', 'true_label': 1, 'predicted_la

In [27]:
_my_dict = {}
def getDict(lista):
    filenames, true, predict, confidence = [], [], [], []
    for diccionario in lista:
        filename = diccionario['filename']
        true_label = diccionario['true_label']
        predicted_label= diccionario['predicted_label']
        confianza = diccionario['max_prob']
        filenames.append(filename)
        true.append(true_label)
        predict.append(predicted_label)
        confidence.append(confianza)
    return filenames, true, predict, confidence

filenames, true, predict, confidence = getDict(lista = results)


In [28]:
_my_dict['filenames'] = filenames
_my_dict['true_labels'] = true
_my_dict['predicted_label'] = predict
_my_dict['confianza'] = confidence

In [29]:
import pandas as pd

df = pd.DataFrame(_my_dict)
print(len(df))

10056


In [30]:
df.sort_values(by="filenames", ascending=True)

,filenames,true_labels,predicted_label,confianza
6224,0.jpg,1,0,0.537655
9385,1.jpg,1,0,0.537637
109,10.jpg,1,0,0.537628
7196,10000.jpg,1,0,0.537632
6743,10001.jpg,1,0,0.537644
...,...,...,...,...
4457,9994.jpg,1,0,0.537625
2675,9995.jpg,1,0,0.537655
2470,9996.jpg,1,0,0.537668
9554,9997.jpg,1,0,0.537663


In [31]:
df_filtrado = df[(df['true_labels'] != df['predicted_label']) | (df['confianza'] <= 0.95)]
tamaño = len(df_filtrado) 

In [32]:
print(tamaño)
df_filtrado

10056


,filenames,true_labels,predicted_label,confianza
0,1343.jpg,1,0,0.537641
1,5679.jpg,1,0,0.537649
2,5577.jpg,1,0,0.537658
3,5377.jpg,1,0,0.537637
4,5974.jpg,1,0,0.537625
...,...,...,...,...
10051,1763.jpg,1,0,0.537632
10052,6499.jpg,1,0,0.537652
10053,3887.jpg,1,0,0.537661
10054,11342.jpg,1,0,0.537646


In [33]:
def deleteConfuseData(directory, dataFrame):
    contador = 0
    general_path = rf'..\..\PetImages\{directory}'
    for idx, row in dataFrame.iterrows():
        filename = row['filenames']
        img_path = os.path.join(general_path, filename)
        try:
            os.remove(img_path)
            contador += 1
        except FileNotFoundError:
            print("Error al encontrar el archivo, ya ha sido removido o no se ha encontrado en la carpeta")

    print(f'\n--- Proceso Finalizado ---')
    print(f'Total de archivos eliminados de {directory}: {contador}')

In [34]:
deleteConfuseData(directory="Dog", dataFrame=df_filtrado)


--- Proceso Finalizado ---
Total de archivos eliminados de Dog: 10056


In [35]:
df_filtrado[df_filtrado['true_labels'] != df_filtrado['predicted_label']]

,filenames,true_labels,predicted_label,confianza
0,1343.jpg,1,0,0.537641
1,5679.jpg,1,0,0.537649
2,5577.jpg,1,0,0.537658
3,5377.jpg,1,0,0.537637
4,5974.jpg,1,0,0.537625
...,...,...,...,...
10051,1763.jpg,1,0,0.537632
10052,6499.jpg,1,0,0.537652
10053,3887.jpg,1,0,0.537661
10054,11342.jpg,1,0,0.537646


In [36]:
df[df['filenames'] == '2064.jpg']

,filenames,true_labels,predicted_label,confianza


Intento de implementación de grad-cam

In [37]:
model.parameters

<bound method Module.parameters of Modelo(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (conv5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv6): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1))
  (conv7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1))
  (conv8): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1))
  (maxpool): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (avg): AdaptiveAvgPool2d(output_size=(1, 1))
  (dense1): Linear(in_features=128, out_features=64, bias=True)
  (dense2): Linear(in_features=64, out_features=2, bias=True)
)>

ESTO NO SÉ POR QUÉ LO HICE JAJAJAJA SI ESTO ES PARA RECONOCIMIENTO FACIAL, PERO ESTE MODELO NI SIRVE PA' ESO  JAJAJAJA XD

In [38]:
#print(f"Modelo antes de dcapitar: {model.dense1} / {model.dense2}")

#model.dense1 = nn.Identity()
#model.dense2 = nn.Identity()

#print(f"Modelo después de decapitar: {model.dense1} / {model.dense2}")

In [39]:
print(model.parameters)

<bound method Module.parameters of Modelo(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (conv5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv6): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1))
  (conv7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1))
  (conv8): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1))
  (maxpool): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (avg): AdaptiveAvgPool2d(output_size=(1, 1))
  (dense1): Linear(in_features=128, out_features=64, bias=True)
  (dense2): Linear(in_features=64, out_features=2, bias=True)
)>


In [40]:
import cv2, numpy as np

def gradCam(model, target_class:int, datos:dict, device):
    #Primero lo ponemss en evalcuación
    model.to(device)
    model.eval()
    activaciones, gradientes = None, None
    
    Data_to_Steal = {"Activations":None, "Gradients":None}
    #Definimos nuestros ganchos a la capa convolucional para espiar y robar las activaciones, así como los gradientes de los mapas de activación.
    def hook_activaciones(modulo, input, output):
        Data_to_Steal["Activations"] = output.detach()

    def hook_gradientes(modulo, input, output):
        Data_to_Steal['Gradients'] = output[0].detach()

    #Ahora tenemos que obtener la capa objetivo
    target_layer = model.layer4[-1]
    ActivationHook = target_layer.register_forward_hook(hook_activaciones)
    GradientHook = target_layer.register_backward_hook(hook_gradientes)

    #Ahora hacemos una predicción con una imagen de gato
    GANGTEL = pd.DataFrame(datos)
    RandomDataSet = customDSAlone(dataframe=GANGTEL, img_path=r'..\..\Petimages', transform=transformador)
    image_DataLoader = DataLoader(dataset=RandomDataSet, batch_size=1)

    data_iter = iter(image_DataLoader)
    image, label, filename = next(data_iter)

    image = image.to(device)
    label = label.to(device)

    print(f"La imagen está en {image.device} y la etiqueta en {label.device}\n Nota: Si es cuda:0  es que está en GPU, sino está en la CPU ")

    #Ahora hacemos una predicción con esa imagen:
    prediccion = model(image) #Aquí se roban las activaciones gracias a los "espias" o hooks

    #Ahora para robar los gradientes de la capa convolucional 8

    """Primero definimos lo que será la capa de salida, para hacer el backprop"""
    target_score = prediccion[0, target_class] #Como solamente le pasamos un solo ejemplo, la matriz de salida será de la forma 1 x 2, donde las filas son los ejemplos y las columnas los logits de cada clase [0.3, 0.13l4]

    model.zero_grad() #Calculo de gradientes
    target_score.backward(retain_graph=True) #Ahora calculamos los gradientes con respecto a la neurona objetivo (la de gato), si quisieras ver el por qué decidió que era perro o así, cambia la neurona de salida

    #Ahora ya que robamos los gradientes y activaciones podemos hacer lo que menciona el paper de grad-CAM v2 https://arxiv.org/pdf/1610.02391
    G = Data_to_Steal['Gradients']
    A = Data_to_Steal['Activations']

    #Estps son los pesos de las activaciones, dando un tensor [1, 128, 1, 1]
    pesos = torch.mean(G, dim = [2, 3], keepdim=True)
    print(f"Forma de los pesos: {pesos.shape}")

    #Ahora que tenemos los pesos, debemos saber que tanto influyeron en las activaciones.
    PesosA = A * pesos
    print(PesosA.shape)

    #Ahora hacemos la suma ponderada de los pesos de cada activación
    heatmap = torch.sum(PesosA, dim=1)
    print(heatmap.shape)

    #Ahora aplicamos ReLu porque solo queremos lo positivo, lo negativo no nos interesa para nada.
    heatmap = torch.nn.functional.relu(heatmap)

    #Ahora normalizamos entre -1 y 1
    print(heatmap.max())

    heatmap -= heatmap.min()
    heatmap /= heatmap.max()

    #Ahora lo metemos todo a CPU, así como quitar el batch para hacerlo un arreglo de NumPy
    print(f"Antes de ponerlo chulo: {heatmap.shape}")
    heatmap = heatmap.squeeze(0).cpu().numpy()
    print(f"Después de arreglarlo un poco xd: {heatmap.shape}")

    #Ahora debemos hacer todo lo de mostrar la imagen con Cv2 bruh
    label = GANGTEL.iloc[0, 1] #Accedo a la fila del indice 0 y obtengo el elemento de la segunda columna "label"
    print(f"Etiqueta: {label}, Nombre: {filename[0]}") #Nomás pa' checar jaja, pero como vimos hace un rato hicimos una lista, por lo que es una lista, así que accedo a el elemento con [0]
    img = cv2.imread(rf'..\..\PetImages\{label}\{filename[0]}') #Ahora que tenemos la ruta, leemos la imagen
    img = cv2.resize(img, (128, 128)) #La ajustamos al tamaño 128, 128

    #Acá también ajustamos el mapa de calor xd
    heatmap = cv2.resize(heatmap, (128, 128))
    heatmap = cv2.applyColorMap(np.uint8(255 * heatmap), cv2.COLORMAP_JET)

    #Superponemos el mapa de calor sobre la imagen
    img = (heatmap * 0.4) + img
    img = np.uint8(img)

    NumArchivo = filename[0] #"10.jpg"
    NumArchivo = NumArchivo.split(".") #['10', '.jpg']
    NumArchivo = NumArchivo[0] #10
    print(NumArchivo)
    cv2.imwrite(f'GradCAMS/GradCAM{NumArchivo}_{label}.jpg', img)

    ActivationHook.remove
    GradientHook.remove

    print("La imagen se ha guardao' tio chaval oshtia safjksahfkashfkas")

Viendo los mapas de gradientes, el modelo mío aprendió mucho de características irrelevantes, por lo que después de haber limpiado un poco el dataset, vamos a reentrenarlo: "ahhh veremos que sucede, estoy un poco nervioso".

In [41]:
#Después de la limpieza reducimos mucho el dataset, por lo que hay desequilibrio de clases entre perros y gatos
total = len(DogDF) + len(CatDF)
PerDog = (len(DogDF) * 100)/total
PerCat = (len(CatDF) * 100)/total
difference = np.absolute(PerDog - PerCat)
print(f"Porcentaje de perros: {PerDog}%")
print(f"Porcentaje de gatos: {PerCat}%")
print(f"Con una diferencia de: {difference}%")
print(len(CatDF))
print(len(DogDF))

Porcentaje de perros: 54.87285823420277%
Porcentaje de gatos: 45.12714176579723%
Con una diferencia de: 9.745716468405547%
8270
10056


In [42]:
class Mendicant_Biasv2(nn.Module):
    def __init__(self):
        super(Mendicant_Biasv2, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=(3, 3))
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=(3, 3))
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=(3, 3))
        self.conv4 = nn.Conv2d(in_channels= 32, out_channels=64, kernel_size=(3, 3))
        self.conv5 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=(3, 3))
        self.conv6 = nn.Conv2d(in_channels=128, out_channels=128, kernel_size=(3, 3))
        self.maxpool = nn.MaxPool2d((2, 2))
        self.dropoutC = nn.Dropout2d(p=0.2)
        self.dropoutL = nn.Dropout(p=0.45)

        self.avg = nn.AdaptiveAvgPool2d((1, 1)) # batch, 128, 1, 1

        self.dense1 = nn.Linear(in_features=128, out_features=64)  
        self.dense2 = nn.Linear(in_features=64, out_features=2)

    def forward(self, x):
        #Bloque convolcuional 1
        x = f.relu(self.conv1(x)) # 126
        x = f.leaky_relu(self.maxpool(self.conv2(x))) # 124 -> Pool = 62
        x = self.dropoutC(x)

        #Bloque convolucional 2
        x = f.relu(self.conv3(x)) # 60
        x = f.leaky_relu(self.maxpool((self.conv4(x)))) # 58 / 2 = 29
        x = self.dropoutC(x)

        #Bloque convolucional 3
        x = f.relu(self.conv5(x)) #27
        x = f.leaky_relu(self.maxpool(self.conv6(x))) # 25 / 2 = 12
        x = self.dropoutC(x)

        x = self.avg(x)
        x = torch.flatten(x, 1)

        x = f.relu(self.dense1(x))
        x = self.dropoutL(x)
        x = self.dense2(x)

        return x

DataAugmentationCats = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.RandomHorizontalFlip(p=0.3),
    transforms.RandomRotation(degrees=10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

DataAugmentationDogs = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.RandomHorizontalFlip(p=0.3),
    transforms.RandomRotation(degrees=10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

trainDataAugmentation = {
    "Cat": DataAugmentationCats,
    "Dog": DataAugmentationDogs
}

Mendicant_bias = Mendicant_Biasv2()

In [43]:
class CustomDS2(Dataset):
    def __init__(self, dataframe, images_dir, transform=None, target_transform=None):
        self.labels = dataframe
        self.directory = images_dir
        self.transform_data = transform
        self.target_transform = target_transform
        self.mapping = {"Cat": 0, "Dog": 1}

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        label_df = self.labels.iloc[idx, 1]
        img_path = os.path.join(self.directory, self.labels.iloc[idx, 0]) 
        filename = self.labels.iloc[idx, 0]

        img = Image.open(img_path).convert('RGB')
        img_label = self.mapping[label_df]

        if self.transform_data: 
            if isinstance(self.transform_data, dict):
                transformacion = self.transform_data.get(label_df, None)
                if transformacion:
                    img = transformacion(img)
            else:
                img = self.transform_data(img)
        if self.target_transform: 
            img_label = self.target_transform(img_label)
        return img, img_label, filename

In [44]:
#Generación de los datasets customizados a mis datos XD LMAO VAMO' A VER QUE PASAAAAAAA
train_dataset = CustomDS2(dataframe=train_df, images_dir=general_path, transform=trainDataAugmentation) #lO que psue antes wachooooo
val_dataset = CustomDS2(dataframe=val_df, images_dir=general_path, transform=transformador) #Esto es lo normal XD
dev_dataset = CustomDS2(dataframe=dev_df, images_dir=general_path, transform=transformador) #Normal

#Generción de los DataLoaders
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=64, shuffle=True)
dev_dataloader = DataLoader(dev_dataset, batch_size=64, shuffle=True)

In [45]:
from datetime import datetime

tiempo = datetime.now()
dia = tiempo.date()

def train_model(modelo, dataloaders:list, optimizador, lossFunction, reduce_lr):
    """
    Aquí el dataloader es [train_dataloader, val_dataloader], por lo que si queremos acceder al primer dataloader, debemos usar indices
    """
    if modelo and isinstance(dataloaders, list):
        modelo = modelo.to(device)

        best_accuracy = 0.0
        
        train_dataloader, val_dataloader = dataloaders
        epochs = 30
        
        print("Iniciando entrenamiento...")

        for i in range(1, epochs + 1):
            #Declaramos las métricas de la epoch
            actual_loss = 0.0
            actual_valloss = 0.0
            running_accuracy = 0.0
            total = 0
            
            modelo.train() # Ponemos el modelo en modo entrenamiento
            for image, label, filename in train_dataloader:
                image = image.to(device)
                label = label.to(device)

                # 1. Limpiar gradientes
                optimizador.zero_grad()

                # 2. Forward pass
                output = modelo(image)
                loss = lossFunction(output, label)

                # 3. Backward pass
                loss.backward()
                optimizador.step()

                # 4. Acumular loss (usando .item())
                actual_loss += loss.item()

            # Promedio de loss de entrenamiento de esta época
            train_loss = actual_loss / len(train_dataloader)

            modelo.eval() # Ponemos el modelo en modo evaluación
            with torch.no_grad():
                for image, label, filename in val_dataloader:
                    image = image.to(device)
                    label = label.to(device)
                    
                    predicts = modelo(image)
                    val_loss_fn = lossFunction(predicts, label)
                    
                    # Acumular loss de validación
                    actual_valloss += val_loss_fn.item()
                    
                    # Calcular accuracy
                    _, predict = torch.max(predicts, 1)
                    total += label.size(0)
                    running_accuracy += (predict == label).sum().item()
            
            # Promedio de loss de validación y accuracy total
            val_loss = actual_valloss / len(val_dataloader)
            reduce_lr.step(val_loss)
            accuracy = (100 * running_accuracy / total)

            # Guardar el mejor modelo
            if accuracy > best_accuracy:
                torch.save(modelo.state_dict(), fr'Models/ModeloPrueba{dia}.pth')
                best_accuracy = accuracy
                print(f"--> Nuevo mejor modelo guardado con acc: {best_accuracy:.2f}%")
            
            print('Vuelta', i, 'Training Loss es: %.4f' %train_loss, 'Validation Loss es: %.4f' %val_loss, 'Precisión es: %.2f %%' % (accuracy))

In [46]:
function_loss = nn.CrossEntropyLoss()
optimizador = torch.optim.Adam(params=Mendicant_bias.parameters(), lr=1e-4, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizador, mode="min", factor=0.1, patience = 2)
#train_model(modelo = Mendicant_bias, dataloaders=[train_dataloader, val_dataloader], optimizador = optimizador, lossFunction=function_loss, reduce_lr=scheduler)

In [47]:
Mendicant_bias.load_state_dict(torch.load(f="./Models/ModeloPrueba2025-11-19.pth", weights_only=True))
results = MismatchLabeling(model = Mendicant_bias, dataloader=dev_dataloader, device = device, FunctionLoss = function_loss)
print(results[0])

El valor de pérdida del mdoelo es: 0.4689, con una precisión de: 78.4741


In [48]:
A101ENELEXPRESSO = pd.DataFrame(results[1])
A101ENELEXPRESSO

,filename,true_label,predicted_label,max_prob
0,753.jpg,0,0,0.787594
1,16350.jpg,1,1,0.855544
2,4047.jpg,0,0,0.833712
3,421.jpg,0,1,0.544413
4,55.jpg,0,0,0.936238
...,...,...,...,...
362,4871.jpg,0,1,0.585337
363,10092.jpg,1,1,0.743383
364,16839.jpg,1,1,0.895656
365,12502.jpg,1,1,0.928214


In [49]:
A101ENELEXPRESSO[A101ENELEXPRESSO['filename'] == '7775.jpg']


,filename,true_label,predicted_label,max_prob
140,7775.jpg,0,0,0.941395


In [66]:
def predict(model, dataloader):
    model.eval() # Ponemos el modelo en modo evaluación
    model.to(device)
    
    # Inicializamos variables por si el dataloader estuviera vacío (seguridad)
    confianza = None
    predict_label = None

    with torch.no_grad():
        for image, label, filename in dataloader:
            image = image.to(device)
            # label = label.to(device) # No la necesitas para predecir

            predicts = model(image)
            prob = torch.nn.functional.softmax(predicts, 1)
            
            # CORRECCIÓN AQUÍ: Agregamos dim=1
            confianza, predict_label = torch.max(prob, 1)
            
            # Como batch_size=1, podemos retornar inmediatamente el primer resultado
            # Usamos .item() para devolver números de Python, no tensores
            return confianza.item(), predict_label.item()
            
    return 0.0, -1 # Retorno por defecto si falla algo

In [51]:
data_img = {
    "filename":["7790.jpg"],
    "label" : ["Cat"]
}
data_img_df = pd.DataFrame(data=data_img)

img_ds = CustomDS2(dataframe=data_img_df, images_dir = r'..\..\PetImages\Cat', transform=transformador, target_transform=None)
img_dl = DataLoader(dataset=img_ds, batch_size=1, shuffle=False)

In [52]:
confianza, predicted_label = predict(model=Mendicant_bias, dataloader=img_dl)

In [53]:
confianza, predicted_label

(0.9295932650566101, 0)

In [54]:
#gradCam(model=Mendicant_bias, target_class = 0, datos=data_img, device = device) #Target class es la neurona con la cual el gradiente será respecto a ella, 0 - Gato; 1 - Perro 

In [55]:
def conv3x3(in_channels, out_channels, stride = 1):
    return nn.Conv2d(in_channels, out_channels, kernel_size=3, stride = stride, padding = 1, bias = False)

class Residual_block(nn.Module):
    def __init__(self, in_channels, out_channels, stride = 1, downsample=None):
        super(Residual_block, self).__init__()
        self.conv1 = conv3x3(in_channels, out_channels, stride)
        self.bn1 = nn.BatchNorm2d(out_channels, affine=True)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(out_channels, out_channels)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = downsample
    
    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        if self.downsample:
            residual = self.downsample(x)
        out += residual
        out = self.relu(out)
        return out

class Mendicant_Biasv3(nn.Module):
    def __init__(self):
        super(Mendicant_Biasv3, self).__init__()
        # Conv = (input - kernel + 2 * padding) / stride

        self.in_channels = 64

        #Primer reducción agresiva de ResNet
        self.stem = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=64, kernel_size=(7, 7), stride=(2, 2), bias=False),
            nn.BatchNorm2d(num_features=64, affine=True),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2)   
        )

        #Bloques residuales
        self.layer1 = self._make_layer(out_channels=64, blocks = 2, stride = 1)

        self.layer2 = self._make_layer(out_channels=128, blocks = 2, stride = 2)
        
        self.layer3 = self._make_layer(out_channels=256, blocks = 2, stride = 2)

        self.layer4 = self._make_layer(out_channels=512, blocks = 2, stride = 2)

        #Promedio
        self.avg = nn.AdaptiveAvgPool2d((1, 1))

        #Capas de clasificación
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=512, out_features=256), 
            nn.BatchNorm1d(256),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.35),
            nn.Linear(in_features=256, out_features=2),
        )

    
    def _make_layer(self, out_channels, blocks, stride=1):
        downsample = None

        if (stride != 1) or (self.in_channels != out_channels):
            downsample = nn.Sequential(
                nn.Conv2d(self.in_channels, out_channels, kernel_size=1, stride=2),
                nn.BatchNorm2d(out_channels)
            )

        layers = []

        layers.append(Residual_block(self.in_channels, out_channels, stride, downsample))

        self.in_channels = out_channels

        for _ in range(1, blocks):
            layers.append(Residual_block(self.in_channels, out_channels))

        return nn.Sequential(*layers)
    
    def forward(self, x):
        x = self.stem(x)
        
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        
        x = self.avg(x)
        x = self.classifier(x)
        return x


Mendicant_biasv3 = Mendicant_Biasv3()

In [56]:
function_loss = nn.CrossEntropyLoss()
optimizador = torch.optim.Adam(params=Mendicant_biasv3.parameters(), lr=1e-4, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizador, mode="min", factor=0.1, patience = 3)
#train_model(modelo=Mendicant_biasv3, dataloaders = [train_dataloader, val_dataloader], optimizador = optimizador, lossFunction=function_loss, reduce_lr=scheduler)

In [57]:
Mendicant_biasv3.load_state_dict(torch.load('./Models/ModeloPrueba2025-11-28.pth', weights_only=True))
results = MismatchLabeling(model = Mendicant_biasv3, dataloader=dev_dataloader, device = device, FunctionLoss = function_loss)
print(results[0])


El valor de pérdida del mdoelo es: 0.1197, con una precisión de: 96.4578


In [65]:

gradCam(model=Mendicant_biasv3, target_class=0, datos = {"filename" : ['Guero.jpg'], "label" : ['Cat']}, device=device)

La imagen está en cuda:0 y la etiqueta en cuda:0
 Nota: Si es cuda:0  es que está en GPU, sino está en la CPU 
Forma de los pesos: torch.Size([1, 512, 1, 1])
torch.Size([1, 512, 4, 4])
torch.Size([1, 4, 4])
tensor(0.3124, device='cuda:0')
Antes de ponerlo chulo: torch.Size([1, 4, 4])
Después de arreglarlo un poco xd: (4, 4)
Etiqueta: Cat, Nombre: Guero.jpg
Guero
La imagen se ha guardao' tio chaval oshtia safjksahfkashfkas


c:\Users\PC\Desktop\Road_to_Hackathon\env\Lib\site-packages\torch\nn\modules\module.py:1868: FutureWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  self._maybe_warn_non_full_backward_hook(args, result, grad_fn)


In [59]:
Mendicant_biasv3.parameters

<bound method Module.parameters of Mendicant_Biasv3(
  (stem): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer1): Sequential(
    (0): Residual_block(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): Residual_block(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track

In [74]:
data = {
    "filename": ["Fredo.jpg"],
    "label": ['Dog']
}

transformador = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

df = pd.DataFrame(data)
ds = CustomDS2(df, r"..\..\Petimages\Dog", transformador)
dl = DataLoader(ds, batch_size=1, shuffle=False)
predict(Mendicant_biasv3, dl)

(0.9985999464988708, 1)

Viéndolo con mi mascota la cual es un Corgi de Gáles, el cual no estuvo en ningún momento en el Dataset, ni de entrenamiento, ni de validación, ni de development, obtuvo una calificación de aproximadamente 99.85% y mostrando su GradCAM, el modelo obtuvo métricas relativamente buenas.

![GradCAM de mi mascota "Fredo"](\GradCAMS\GradCAMfredo_Dog.jpg)